In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree

import requests
from lxml import html


import re
import hashlib
import sqlite3
import telegram_send
import telebot
import time
import configparser


In [2]:
#Ограничивает длину слов
WORLD_LIMIT = 400

In [3]:
# Создание объекта конфигурации
config = configparser.ConfigParser()

# Чтение данных из файла
config.read('config.ini')

BOT_TOKEN = config.get('tokens', 'BOT_TOKEN')
CHAT_ID = config.get('tokens', 'CHAT_ID')

## Функции

In [4]:
def get_div(tree, xpath_format, div_mode=True):
    list_divs = []


    # Выбор элемента
    selected_element = tree.xpath(xpath_format)


    for div in selected_element:
        # Получение HTML-разметки только для выбранного элемента
        html_representation = html.tostring(div, encoding='unicode', method='html')

        list_divs.append(html_representation)
    return list_divs


# Parse

In [5]:
dict_mus = {
    
'Музей победы': ['https://victorymuseum.ru/playbill/',
'//div[@class="oblique-items"]//div[@class="item"]', 'top_museum'],
    
'Музей блокады Ленинграда': ['https://blokadamus.ru/музей-обороны-и-блокады-ленинграда/события/',
'//*[@class="elementor-post__card"]'],
    
'Музей минобороны в Москве': ['http://www.cmaf.ru/',
'//*[@class="reg-news"]//p[@class="news-item"]/following-sibling::*[1]'],
    
'Музей дорогами победы (парк Патриот)': ['https://1418museum.ru/news/',
'//*[@class="newsItem__contains"]'],
    
'Бородинская битва': ['https://1812panorama.ru/afisha',
'//*[@class="field-more-title"]'],
    
'Музей современной истории России': ['https://sovrhistory.ru/exhibitions/future/index.php',
'//*[@class="exhibitions__text"]'],
    
'Выставочный центр Манеж, Москва': ['https://moscowmanege.ru/ru/vystavki-all/',
'//*[@class="news-block exhibition"]'],
    
'Музей Москвы': ['https://mosmuseum.ru/exhibitions/',
'//*[@class="articles-grid"]//*[@class="content"]'],
    
'ГИМ, Москва': ['https://mediashm.ru/?all=1', '//a[@class="element"]'],
    
'Музей кино, Москва': ['https://www.museikino.ru/media_center/',
'//div[@class="media__inf"]'],
    
'Музей военной истории «Свердловск: Говорит Москва!» ': ['https://ugvim.ru/news/',
'//a[@class="front_cont h_more_w"]'],
    
'Дом-музей В.И. Ленина': ['https://www.leninmemorial.ru/news/',
'//div[@class="row row-news-page"]'],
    
'Мемориал «В честь героев Курской битвы»': ['http://мемориал.бгикм.рф/меню/новости',
'//div[@class="post"]'],
    
'Мемориальный музей военного и трудового подвига 1941–1945 гг': ['http://patriotrm.clan.su/news/?page1',
'//td[@style="padding:3px;"]'],
    
'ГОСУДАРСТВЕННЫЙ ВОЕННО-ИСТОРИЧЕСКИЙ МУЗЕЙ-ЗАПОВЕДНИК «ПРОХОРОВСКОЕ ПОЛЕ»': ['https://прохоровское-поле.рф/vystavochnaya-deyatelnost.html',
'//div[@class="news-item"]'],
    
'Музейно-мемориальный комплекс «История танка Т-34»': ['https://museum-t-34.ru/новости/',
'//div[@class="site_content_left"]'],
    
'Музей «Боевая слава» г. Дербента': ['https://derbentmuseum.ru/',
'//section[@class="blog"]//div[@class="col-lg-4"]'],
    
'Музей «Мемориал Победы»': ['https://memorial24.ru/novosti',
'//div[@class="novelty clearfix novelty-teaser"]'],
    
'Музей «Смоленщина в годы Великой Отечественной войны 1941−1945 гг.': ['https://www.smolensk-museum.ru/novosti/muzeynaya_zhizn/',
'//div[@class="news__item"]'],
    
'Музей «Невский пятачок': ['https://www.lenoblmus.ru/news',
'//li[@class="col-md-4 col-lg-3 col-sm-6 col-xs-12"]'],
    
'Музей «Дорога жизни»': ['https://navalmuseum.ru/news',
'//div[@class="content-box"]'],
    
'Самарский областной историко-краеведческий музей им. П.В. Алабина': ['https://alabin.ru/',
'//div[@class="news__post"]'],
    
'Якутский государственный объединенный музей истории и культуры народов Севера им. Ем. Ярославского': ['https://yakutmuseum.ru/',
'//a[@class="news-item"]'],
    
'Краснодарский государственный историко-археологический музей-заповедник им. Е.Д. Фелицына': ['https://felicina.ru/news/',
'//div[@class="content-area content-left col-md-8 col-sm-8"]//article'],
    
'Государственный исторический музей Южного Урала': ['http://www.chelmuseum.ru/news/#content',
'//div[@class="card"]'],
    
'Тамбовский областной краеведческий музей': ['https://tmbmuseum.ru/mainmenu/news/relevant',
'//h3'],
    
'Национальный музей Республики Башкортостан ': ['https://museumrb.ru/category/afisha/events/',
'//div[@class="upcoming-event-2 pt-80 pb-80"]//div[@class="single-event single-event-3 mb-30 wow fadeInUp my-single-event-3"]'],
    
'Ставропольский государственный музей-заповедник': ['https://stavmuseum.ru/',
'//div[@class="anonsiblock"]'],
    
'Новосибирский государственный краеведческий музей': ['https://youmuseum.ru/news/',
'//div[@class="text"]'],
    
'Оренбургский губернаторский историко-краеведческий музей ': ['http://ogikm.ru/',
'//div[@class="tab-content"]//div[@class="item"]'],
    
'Омский государственный историко-краеведческий музей': ['https://sibmuseum.ru/news/',
'//a[@class="news-card__link"]'],
    
'Национальный музей Удмуртской Республики им. К. Герда': ['https://nmur.ru/afisha',
'//div[@class="single-block"]'] ,
    
'Саратовский областной музей краеведения': ['http://www.comk.ru/museum/news/',
'//div[@class="news-item"]'] ,   

'Национальный музей Республики Марий Эл им. Т. Евсеева ': ['https://www.fumus.ru/events/',
'//ul[@class="events__list"]/li'] ,  
    
'Ростовский областной музей краеведения ': ['https://www.rostovmuseum.ru/calendar/list-of-events/',
'//div[@class="row"]//div[@class="s-news-wrapper"]'] ,  

'Воронежский областной краеведческий музей ': ['https://museum-vrn.ru/muzej/novosti/',
'//a[@class="i-wrap"]'] , 
    
'Национальный музей Республики Татарстан ': ['https://tatmuseum.ru/news/?PAGEN_1=4',
'//a[@class="news-item"]'] , 

'Брянский государственный краеведческий музей ': ['https://bgkm.ru/',
'//div[@class="bloglatestnews1"]/div'] , 
    
'Музей истории Бурятии им. М.Н. Хангалова ': ['https://muzeyrb.ru/sobytiya-1?page=2',
'//article[@class="col-lg-4 mb-50 animate-conner"]'] , 
    
'Национальный музей Республики Дагестан им. А. Тахо-Годи ': ['https://dagmuzey.ru/events',
'//a[@class="s-events-wrapper__item s-events-wrapper-item"]'] , 
    
'Ульяновский областной краеведческий музей имени И.А. Гончарова ': ['https://uokm.site/category/уокм/новости/',
'//a[@class="a news-link"]'], 
    
'Музей истории города Улан-Удэ ': ['https://uumuseum.ru/news/',
'//a[@class="event-index-item"]'], 

'Национальный музей Республики Коми ': ['https://museumkomi.ru/?cat=3',
'//div[@class="special-category"]'], 

'Национальный музей Республики Алтай имени А.В. Анохина ': ['http://musey-anohina.ru/index.php/novosti',
'//div[@class="catItemBody"]'], 
    
'Чувашский национальный музей ': ['https://chnmuseum.ru/news/',
'//main[@class="site-main"]/article'],   
    
'Мордовский республиканский объединенный краеведческий музей им. И.Д. Воронина ': ['http://mrkm.ru/novosti/?PAGEN_1=4',
'//a[@class="box news"]'],  
    
'Национальный музей им. Алдан Маадыр Республики Тыва ': ['https://музейтыва.рф/?cat=16&paged=1',
'//div[@class="main col-lg-9 col-md-8  postlist"]/article'], 

'Хабаровский краевой музей имени Н.И. Гродекова ': ['https://hkm.ru/news/',
'//a[@class="new-item list-item"]'], 
    

'Калининградский областной историко-художественный музей ': ['https://koihm.ru/category/novosti/',
'//div[@class="article-inner"]'], 
    
'Национальный музей Республики Карелия ': ['https://kareliamuseum.ru/news/',
'//article[@class="clearfix l-news-article"]'], 
    
'Астраханский музей заповедник ': ['https://astmuseum.ru/ru/news/',
'//div[@class="iblock-list-item-wrap  "]'], 
    
'Сургутский краеведческий музей ': ['https://skmuseum.ru/',
'//ul[@class="is-layout-flow wp-block-post-template has-small-font-size"]/li'], 

'Магаданский областной краеведческий музей ': ['https://magadanmuseum.ru/index.php?do=cat&category=news',
'//div[@id="dle-content"]/table'], 

'Курский областной краеведческий музей ': ['http://kursk-museum.ru/',
'//div[@class="clearfix news_content"]'],  
    
'Национальный музей Республики Адыгея ': ['https://adyg-museum.ru/ru/',
'//div[@class=" col-12 col-sm-12 col-md-6 col-lg-4 col-xl-4"]|//div[@class=" col-12 col-sm-12 col-md-12 col-lg-6 col-xl-6"]'],  

'Музейно-выставочный центр г. Иваново ': ['http://www.igikm.ru/sobytiya/novosti/',
'//p[@class="announce-text"]'], 
    
    
    
  
                                                   }

In [6]:
dict_info = {}


headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
}

for name in dict_mus:
    site = dict_mus[name][0]
    xpath = dict_mus[name][1]
    
    response = requests.get(site,  verify=False
                            , headers=headers
                           )

    if response.status_code == 200:
        # Создаем объект дерева HTML из содержимого ответа
        tree = html.fromstring(response.text)
        
        
        target_obj = get_div(tree, xpath)  
        dict_info[site] = target_obj
        
        
    else:
        print(f'Ошибка статуса {response.status_code} для {site}')

In [7]:
df_raw_p = pd.DataFrame([dict_info]).T

In [8]:
df_raw_p = df_raw_p.explode(0)
df_raw_p = df_raw_p.reset_index()

In [9]:
df_raw_p = df_raw_p.rename(columns={'index':'site', 0:'raw_str'})

In [10]:
df_raw_p[df_raw_p['raw_str'].isna()]

,site,raw_str


In [11]:
df_raw_p = df_raw_p[df_raw_p['raw_str'].notna()].copy()

In [12]:
df_raw_p['text_elem'] = df_raw_p['raw_str'].apply(lambda x: BeautifulSoup(x, 'html.parser').text)

In [13]:
df_raw_p = df_raw_p.drop_duplicates(['site', 'text_elem']).copy()
df_raw_p = df_raw_p.reset_index(drop=True).copy()

## Делаем колоку с хэшом

In [14]:
def generate_hash(input_string):
    # Создаем объект хэша SHA-256
    hash_object = hashlib.sha256()

    # Преобразуем входную строку в байты и обновляем хэш
    hash_object.update(input_string.encode('utf-8'))

    # Получаем шестнадцатеричное представление хэша
    hash_value = hash_object.hexdigest()

    return hash_value

In [15]:
# Колонка для хеширования
df_raw_p.insert(0,'hash_col', df_raw_p['site']+ ' ' + df_raw_p['text_elem'])

df_raw_p['hash_col'] = df_raw_p['hash_col'].apply(generate_hash)

In [16]:
df_raw_p.insert(0,'date_stamp', pd.to_datetime('today'))

## Загружаем в db

In [17]:
conn = sqlite3.connect('museums.db')

In [18]:
# Создаю таблицы

qu = '''
    CREATE TABLE if not EXISTS "parse" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT
)
'''
conn.execute(qu)

qu = '''
    CREATE TABLE if not EXISTS "parse_temp" (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE,
    "site" TEXT,
    "raw_str" TEXT,
    "text_elem" TEXT

)
'''
conn.execute(qu)

# Добавляю таблицу, для успешных выполнений
qu = '''
    CREATE TABLE if not EXISTS send_telegram (
    "date_stamp" TIMESTAMP,
    "hash_col" TEXT UNIQUE, 
    teleg_mes TEXT
)
'''
conn.execute(qu)

In [19]:
# Временная таблица
df_raw_p.to_sql('parse_temp', conn, index=False, if_exists='replace')


# Импортирую значения с новыми значениями по хэшу
qu = '''
INSERT INTO parse
select *
from parse_temp
where hash_col not in (select hash_col from parse)

'''
conn.execute(qu)

In [20]:
qu = '''
select *
from parse
where hash_col not in (select hash_col from send_telegram)
order by date_stamp
'''
df_n_compl = pd.read_sql(qu, conn)

# Чистка

In [21]:
def adj_lim(x:list):
    '''Ограничивает длину слова'''
    for i, el  in enumerate(x):

        if len(el) > WORLD_LIMIT:
            x[i] = el[:WORLD_LIMIT]+'...'
            
    return x



list_unuse = ['\t\t\t\tЧитать далее»\t\t\t', 'подробнее >>', 'Подробнее', 'Заказать', 'подробнее...']


def drop_words(x:list):
    '''
    Удаляю лишние слова 
    '''
    new_list = []
    for l in x:

        if not l in list_unuse and not bool(re.match(r'^[0-9 /]+$', l)) and l:
            new_list.append(l)
#             print('********************')
#             print(l)
#             print('********************')
#         else:
#             print('********************')
#             print('Не лобавлено',l)
#             print('********************')
    return new_list

        

In [22]:
# Убираю \r
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('\r', '')

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('^[(\\n)\\t ]+','', regex=True)

In [23]:
df_raw_p['text_elem'] = df_raw_p['text_elem'].str.replace('[0-9]* Comments', '')

In [24]:
# Сплит строки
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(lambda x: re.split(r"(\n)", x))

# Ограничивает длину
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(adj_lim)

# Удаление лишних слов 
df_n_compl['text_elem'] = df_n_compl['text_elem'].apply(drop_words)

df_n_compl['text_elem']= df_n_compl['text_elem'].apply(lambda x: ''.join(x))

In [25]:
# Удаление \n в конце текста
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n)+$', '', regex=True)

# Замена \n\n на меньшее число
df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace('(\\n){3,}','\n\n', regex=True)

df_n_compl['text_elem'] = df_n_compl['text_elem'].str.replace(' {3,}',' ', regex=True)

# Telegram

## Добавляем имена музеев

In [26]:
d_mus_names = {}

for k in dict_mus:
    d_mus_names[dict_mus[k][0]] = k

In [27]:
df_n_compl['mus_name'] = df_n_compl['site'].map(d_mus_names)

df_n_compl['mus_name']= df_n_compl['mus_name'].fillna(df_n_compl['site'])


## Отправка

### Подготовка

In [28]:
df_n_compl['teleg_mes'] = '[' + df_n_compl.mus_name + ']' + '(' + df_n_compl.site + ')' + '\n'

df_n_compl['teleg_mes'] = df_n_compl['teleg_mes']+ df_n_compl.text_elem

In [29]:
df_n_compl = df_n_compl[df_n_compl['text_elem']!='']

In [30]:
df_n_compl['teleg_mes']

0     [Музейно-выставочный центр г. Иваново ](http:/...
1     [Музейно-выставочный центр г. Иваново ](http:/...
2     [Музейно-выставочный центр г. Иваново ](http:/...
3     [Музейно-выставочный центр г. Иваново ](http:/...
4     [Музейно-выставочный центр г. Иваново ](http:/...
5     [Музейно-выставочный центр г. Иваново ](http:/...
6     [Музейно-выставочный центр г. Иваново ](http:/...
7     [Музейно-выставочный центр г. Иваново ](http:/...
9     [Музейно-выставочный центр г. Иваново ](http:/...
10    [Музейно-выставочный центр г. Иваново ](http:/...
11    [Музейно-выставочный центр г. Иваново ](http:/...
12    [Музейно-выставочный центр г. Иваново ](http:/...
13    [Музейно-выставочный центр г. Иваново ](http:/...
14    [Музейно-выставочный центр г. Иваново ](http:/...
15    [Музейно-выставочный центр г. Иваново ](http:/...
Name: teleg_mes, dtype: object

In [31]:
df_mes = df_n_compl[['hash_col', 'teleg_mes']].copy()

In [32]:
pd.options.display.max_colwidth = 1000

In [33]:
df_mes.shape

(15, 2)

In [34]:
df_mes

,hash_col,teleg_mes
0,cefff7a54ed52c8c3192570eaa94a489dbcfd4145eb394b4d03e2e691c9cea0c,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nВ Музее промышленности и искусства будет проходить новое музейное занятие ""По следам мифических существ"" для детей младшего и среднего школьного возраста."
1,fa7c92bd4c8375998b65ac75fa55a9e18336f8577495002c9b4d2550872b1d0f,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nУважаемые посетители, планируя визит в музей, обращайте внимание на график работы отделов в праздничные дни!"
2,0d43b20d29c9c7b16e19ef9c9c36f69c666287afc9fe1a5774b099131c9f3149,[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nВ праздничные дни Музейно-выставочный центр ждет ивановцев и гостей города. Для них подготовлена программа новогодних мероприятий.
3,b2da6679a3a10e747a2af346b7dc913e1a0de91add1063d9a16915bee30fcc3e,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\n26 декабря 1914 года двери Музея редкостей и древностей Д. Г. Бурылина открылись для посетителей. С самого открытия и до настоящего времени Музей остаётся жемчужиной нашего города, которая хранит в себе историю и культуру прошлых веков."
4,91992f2bf87d199513056459f5446b149f1138686b7816aefcb939bc46e9e5bf,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\n4 января 2024г. в 14:00 авторскую экскурсию по выставке ""Праздничный мир в открытке"" проведет ее куратор Людмила Николаевна Куприянова."
5,449556a2059658b613004fe3a4cdc2c5bcc5c157d9a45aa76d078f63a0d64317,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nБаринов Максим Вячеславович, младший научный сотрудник, принял участие в городском конкурсе для одарённой молодежи «Большие надежды»."
6,2887987a0248a53ed8e557f088cfa83b8b1700b273746aeb56513ccd8c051730,[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\n23 декабря Музейно-выставочный центр закрыт для посетителей.
7,ec6069c905ad68cf1d746891a38fddf01202415ea4ae2305e064feb77f825cf2,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nВ предпраздничные дни - 26, 27 и 28 декабря - Музейно-выставочный центр приглашает на Рождественский лекторий желающих погрузиться в праздничную атмосферу и познакомиться ближе с историей светлого праздника."
9,1ad1ac2322a83c480c53eccafc32461c3e76d7dad6b23b5c24cf00b980fda788,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nНа открытии выставки ""Праздничный мир в открытке"" были проведены экскурсия и мастер-класс"
10,10974532d2d39eeecc6dcbce8d4484d42fe408869a2d3a94dfa11863095dc10c,"[Музейно-выставочный центр г. Иваново ](http://www.igikm.ru/sobytiya/novosti/)\nВ Музее ивановского ситца с 14 декабря начинает свою работу выставка ""Сказочные миры И. Я. Билибина"". В экспозиции будут представлены уникальные иллюстрации, открытки и литографии Ивана Билибина из сборников сказок."


### Исполнение

In [36]:
# Убирайю символ не разрывного пробела
df_mes['teleg_mes'] = df_mes['teleg_mes'].str.replace('\xa0', ' ', regex=True)

In [37]:
execute_session = pd.to_datetime('now')


# Замените 'YOUR_BOT_TOKEN' на токен вашего бота
bot = telebot.TeleBot(BOT_TOKEN)

# Замените 'YOUR_CHAT_ID' на ID чата, куда вы хотите отправить сообщение
chat_id = CHAT_ID

PAUSE_ITER = 20
PAUSE_TIME = 50


# Ожидание перд запуском
# time.sleep(PAUSE_TIME)
for num, tuple_mes in enumerate(df_mes.itertuples()):
    
    try:
        # Отправка сообщения
        bot.send_message(chat_id, tuple_mes.teleg_mes, parse_mode='Markdown')
    
        # Сохранение успешных сообщений
        df = pd.DataFrame([tuple_mes]).drop(columns='Index')

        df['date_stamp'] = execute_session

        if num % 19==0 and num!=0:
            print('Ожидание')
            time.sleep(PAUSE_TIME)

        df.to_sql('send_telegram', conn, if_exists='append', index=False)
    except Exception as send_error:
        print(num, send_error, tuple_mes)
        df_error = pd.DataFrame([[tuple_mes.hash_col, num,send_error, 
                                              tuple_mes.teleg_mes]], columns=['hash_col','num', 'error', 'teleg_mes'])

        df_error.insert(0,'date_time', execute_session)

        df_error.to_sql('send_error', conn, index=False, if_exists='append')

    
    
